In [1]:
import pandas as pd
import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span

from spacy import displacy


import random

In [2]:
#Import 
df = pd.read_excel('../data/combined-training-final.xlsx')

In [3]:
df.head()

,Body
0,This app looks awesome. Very well thought out...
1,This update is terrible basically unusable ple...
2,The majority of the time this app is wonderful...
3,One my favorite apps to use while working. I’m...
4,This absolutely works perfectly without diffic...


In [6]:
#DF styles..

pd.set_option('min_rows', 20)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_colwidth', None)

#df.style.set_properties(** 'text-align' 'left' )

# th_props = [
#   ('font-size', '11px'),
#   ('text-align', 'center'),
#   ('font-weight', 'bold'),
#   ('color', '#6d6d6d'),
#   ('background-color', '#f7f7f9')
#   ]

# # Set CSS properties for td elements in dataframe
# td_props = [
#   ('font-size', '11px'),
#   ('text-align', 'left')
#   ]

# # Set table styles
# styles = [
#   dict(selector="th", props=th_props),
#   dict(selector="td", props=td_props)
#   ]


# (df.style
#     .set_table_styles(styles))

In [7]:
df

,Body
0,This app looks awesome. Very well thought out...
1,This update is terrible basically unusable ple...
2,The majority of the time this app is wonderful...
3,One my favorite apps to use while working. I’m...
4,This absolutely works perfectly without diffic...
5,Just deleted. Crashed. Didn’t update banking. ...
6,Transferring company file from online QuickBoo...
7,I have a Handyman business and this app made t...
8,Very easy to use BUT multi currency is a pain....
9,I was hesitant to change the way I was doing t...


In [8]:
nlp = spacy.load('en_core_web_sm')

df.review = df.Body.str.lower()

print(nlp.pipe_names)

['tagger', 'parser', 'ner']


/home/rmajumder/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
import json

filename = "../data/train_ner.json"
print(filename)


with open(filename) as train_data:
	train = json.load(train_data)

TRAIN_DATA_NEW = []
for data in train:
	ents = [tuple(entity) for entity in data['entities']]
	TRAIN_DATA_NEW.append((data['content'],{'entities':ents}))


with open('{}'.format(filename.replace('json','txt')),'w') as write:
	write.write(str(TRAIN_DATA_NEW))

print('-------------Copy and Paste to spacy training-------------')
print()
print()
print()
# print(TRAIN_DATA_NEW)
print()
print()
print()
print('--------------------------End-----------------------------')

../data/train_ner.json
-------------Copy and Paste to spacy training-------------






--------------------------End-----------------------------


In [5]:

def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.load('en_core_web_sm')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    else:
        ner = nlp.get_pipe("ner")
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp


prdnlp = train_spacy(TRAIN_DATA_NEW, 20)

Statring iteration 0
{'ner': 1868.073740886898}
Statring iteration 1
{'ner': 2197.6306609648163}
Statring iteration 2
{'ner': 1979.859714338042}
Statring iteration 3
{'ner': 1962.2909433038087}
Statring iteration 4
{'ner': 2076.06037901674}
Statring iteration 5
{'ner': 1976.3216170107619}
Statring iteration 6
{'ner': 2027.886404564189}
Statring iteration 7
{'ner': 1850.6757356186986}
Statring iteration 8
{'ner': 1891.149167895685}
Statring iteration 9
{'ner': 1935.1061993621483}
Statring iteration 10
{'ner': 1866.9506283086985}
Statring iteration 11
{'ner': 1811.5043289405826}
Statring iteration 12
{'ner': 1884.372129321292}
Statring iteration 13
{'ner': 1828.1196664247223}
Statring iteration 14
{'ner': 1894.5513830871691}
Statring iteration 15
{'ner': 1736.972285602442}
Statring iteration 16
{'ner': 1772.4584260297638}
Statring iteration 17
{'ner': 1829.3317158145296}
Statring iteration 18
{'ner': 1761.015005559146}
Statring iteration 19
{'ner': 1875.795606687183}


In [6]:
# Save our trained Model
prdnlp.to_disk("qbo_aspect")

In [7]:
#Test your text
test_text = ["Cancelling your subscription seems impossible. I've emailed, called, used their help menu. It shouldn't be this hard. Still trying to cancel.",
              "My App has stopped working on my Samsung s9 for 2 days in a row. You should have sent an email to inform subscribers if your app is going to be experiencing a downtime.",
              "Cannot open customer invoices. Please correct ASAP!",
               "bugs on iOS13 please update ASAP!"]
for text in test_text:
    print('-----')
    doc = prdnlp(text)
    for ent in doc.ents:
        print('keyword=' + ent.text, 'aspect=' + ent.label_)

-----
keyword=called aspect=Quality
-----
keyword=App aspect=App
-----
keyword=invoices aspect=Invoicing
-----
keyword=bugs aspect=Quality
keyword=update aspect=Updates


In [8]:
for index, row in df.iterrows():
    aspect_terms = []
    doc = prdnlp(row['Body'])
    for ent in doc.ents:
        if ent.label_ not in aspect_terms:
            aspect_terms.append(ent.label_)

    df.loc[index,'aspect_terms'] = ", ".join(aspect_terms)

NameError: name 'df' is not defined

In [28]:
df

,Body,aspect_terms
0,"This app looks awesome. Very well thought out design and it appears that most of the functions are right at my fingertips. Not so. I have used it mostly for categorizing expenses. I was trying to run a report, but nothing really made sense. I went first to google to look for some tutorial videos, but it appears that most of the videos available online are only advertisements. That is when I went to the help and support section in app. In the section there is a FAQs area. Perfect! When I read the only article in this section I thought it must be some kind of joke. It was an article on how to get to the FAQs section on the app. It is like some perverted Groundhog Day scene, or like when you are reviewing a resume for a potential new hire and one of the sections is the filler text from the template they created the resume from. I expect more from this company. Fix this or simply remove it.","App, Expenses, Receipt Capture"
1,This update is terrible basically unusable please change it back ASAP,
2,The majority of the time this app is wonderful. On a regular basis it will completely close out and I will lose the transaction I’m in the middle of. \nI’ve continually notified Intuit about this since the problem started in April of 2019. The consistent response is that they close out the ticket same day and never respond or research the issue. \nToday I’m told that they no longer have a department to help customers with the app. \nNow I’ve just recently renewed my annual subscription so I’m out that money because this product is unreliable.,"App, Tax, Quality, Subscription"
3,One my favorite apps to use while working. I’m on it like a social media page. Thanks quick books.,
4,This absolutely works perfectly without difficulty.,
5,Just deleted. Crashed. Didn’t update banking. Five star to one in a flash.,Quality
6,Transferring company file from online QuickBooks to TurboTax is only was possible by transferring the company file to desktop QuickBooks and from there to TurboTax. To do this transition you will need an old computer with windows 7 and IE11 32 Bit. Totally ridiculous and Intuit looks does not care about this issue and left their customers alone with the issue. It’s a shame.,Quality
7,I have a Handyman business and this app made the paperwork side of the business easy!! I definitely recommend,App
8,"Very easy to use BUT multi currency is a pain. I’m based in US but invoice in UK, Mexico and Spain and with QuickBooks you have to be 100% one currency so it’s a pain for my customers and makes me look second rate as a “in global” business","Invoicing, Quality, Accountant"
9,I was hesitant to change the way I was doing things and use quickbooks when we switched to it a few years back. It has literally transformed how we do things in such a positive manner! I look back to how I used to do it and regret not making the switch sooner.,Quality


In [9]:
df.to_csv('../data/auto_trained.csv', index = False)

NameError: name 'df' is not defined

In [10]:
nlp = spacy.load('en_core_web_sm')

In [11]:
doc = nlp(u"Love the Transactions feature, but i dislike the App")
spacy.displacy.render(doc, style='dep', jupyter=True)

In [12]:
qbo_nlp = spacy.load('qbo_aspect')
print(qbo_nlp.pipe_names)

['tagger', 'parser', 'ner']


In [75]:
doc2 = qbo_nlp("This is a disappointing app. The expenses feature is however is very good")

In [76]:
spacy.displacy.render(doc2, style='dep', jupyter=True)

In [77]:
spacy.displacy.render(doc2, style='ent', jupyter=True)

# Aspect sentiment analysis experiment

In [ ]:
for token in sent:
    print(token.text, token.pos_)

In [78]:
from afinn import Afinn
af = Afinn()

In [139]:
def sentiment_text(val): 
    if val > 0.0:
        return "positive"
    elif val < 0.0:
        return "negative"
    else:
        return "neutral"

In [140]:
test_reviews = ["This is a disappointing app. The expenses feature is however is very good", 
                "Log in issues and it stopped tracking my mileage on its on. I'm trying to pull my information for my taxes and it will not let me log on. More than frustrating, I want my money back!!!! And yes I have tried reinstalling the app!",
                "Extremely slow. No it not slow wifi no it's not the device note 10 it's just a slow app",
                "Amazing app, I literally use it everyday. You need this app if you have your own business.",
                "Great App but have had issues with mileage tracking freezing up while I am categorizing. It then kicks me out.",
                "Even though I canceled my subscription 1 month ago, it still continues to withhold money from me.",
                "Useless app. Waste of money. Lousy support. Subscribe at your own risk. I am cancelling mine"
            
               ]

In [138]:
for review in test_reviews:
    doc2 = qbo_nlp(review)

    for sent in doc2.sents:
        print("----")
        score = af.score(sent.text)
        print("sentence: %s, polarity: %s (%s)" %(sent.text, str(score), sentiment_text(score)))
        for ent in sent.ents:
            print("entity: " + str(ent.label_))

----
sentence: This is a disappointing app., polarity: -2.0 (negative)
entity: App
----
sentence: The expenses feature is however is very good, polarity: 3.0 (positive)
entity: Expenses
----
sentence: Log in issues and it stopped tracking my mileage on its on., polarity: -1.0 (negative)
entity: Quality
----
sentence: I'm trying to pull my information for my taxes and it will not let me log on., polarity: 0.0 (neutral)
----
sentence: More than frustrating, I want my money back!!!!, polarity: -1.0 (negative)
----
sentence: And yes I have tried reinstalling the app!, polarity: 1.0 (positive)
entity: Quality
----
sentence: Extremely slow., polarity: 0.0 (neutral)
entity: Experience
----
sentence: No it not slow wifi, polarity: -1.0 (negative)
----
sentence: no it's not, polarity: -1.0 (negative)
----
sentence: the device note 10, polarity: 0.0 (neutral)
----
sentence: it's just a slow app, polarity: 0.0 (neutral)
----
sentence: Amazing app, I literally use it everyday., polarity: 4.0 (posi

# Dependency Parser

In [109]:
doc2 = qbo_nlp(u"Beautiful App. Easy and Smooth")

In [110]:
for sent in doc2.sents:
    print("----")
    score = af.score(sent.text)
    print("sentence: %s, polarity: %s (%s)" %(sent.text, str(score), sentiment_text(score)))
    for ent in sent.ents:
        print("entity: " + str(ent))

----
sentence: Beautiful App., polarity: 3.0 (positive)
entity: App
----
sentence: Easy and Smooth, polarity: 1.0 (positive)
entity: Easy and


In [27]:
spacy.displacy.render(doc2, style='dep', jupyter=True)

In [28]:
doc1 = qbo_nlp(u"This app has a lot of bugs and performance is bad")

In [29]:
spacy.displacy.render(doc1, style='dep', jupyter=True)

In [14]:
entities=[(i, i.label_) for i in doc1.ents]
entities

[(app, 'App'), (bugs, 'Quality')]

In [10]:
spacy.displacy.render(doc1, style='ent', jupyter=True)

In [28]:
spacy.displacy.render(doc1, style='dep', jupyter=True)